<a href="https://colab.research.google.com/github/DaffaHamsy/UTS-SIstem-Cerdas/blob/main/soal_UTS_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import random
import math

# Definisikan koordinat titik-titik pengantaran
customer_coordinates = {
    'a': (15, 5),
    'b': (10, 30),
    'c': (5, 10),
    'd': (15, 20),
    'e': (25, 10),
    'f': (30, 20),
    'g': (45, 5),
    'h': (35, 30),
    'i': (45, 25),
    'j': (40, 10)
}

# Fungsi untuk menghitung jarak antara dua titik
def calculate_distance(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

# Fungsi untuk menghitung total waktu pengantaran dan pendapatan
def calculate_delivery_time_and_income(delivery_sequence):
    total_distance = 0
    total_income = 0
    current_time = 0
    for i in range(len(delivery_sequence) - 1):
        distance = calculate_distance(customer_coordinates[delivery_sequence[i]], customer_coordinates[delivery_sequence[i+1]])
        total_distance += distance
        current_time += distance  # Waktu tempuh dihitung sebagai jarak
        if delivery_sequence[i] == 'a' or delivery_sequence[i] == 'i':
            if current_time <= 180:  # Maksimal 3 jam (180 menit)
                total_income += 15000  # Bonus untuk pelanggan Merah
            else:
                total_income -= 10000 * ((current_time - 180) // 30)  # Denda 10.000 per setiap 30 menit keterlambatan
        elif delivery_sequence[i] == 'e' or delivery_sequence[i] == 'f':
            if current_time <= 120:  # Maksimal 2 jam (120 menit)
                total_income += 15000  # Bonus untuk pelanggan Kuning
            else:
                total_income -= 5000 * ((current_time - 120) // 30)  # Denda 5.000 per setiap 30 menit keterlambatan
    return total_income

# Fungsi untuk menghasilkan populasi awal
def generate_initial_population(population_size, customers):
    initial_population = []
    for _ in range(population_size):
        sequence = customers[:]
        sequence.remove('g')  # Hapus 'g' dari daftar pelanggan sementara
        random.shuffle(sequence)  # Acak urutan pelanggan
        sequence.insert(0, 'g')  # Masukkan 'g' sebagai pelanggan pertama
        initial_population.append(sequence)
    return initial_population

# Algoritma genetik untuk optimasi pendapatan abang delivery
def genetic_algorithm(population_size, mutation_rate, num_generations, customers):
    best_chromosome = None
    best_income = float('-inf')

    population = generate_initial_population(population_size, customers)
    for generation in range(num_generations):
        for chromosome in population:
            income = calculate_delivery_time_and_income(chromosome)
            if income > best_income:
                best_chromosome = chromosome
                best_income = income

    return best_chromosome, best_income

# Main program
population_size = 50
mutation_rate = 0.1
num_generations = 100
customers = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']  # Include 'g'

best_chromosome, total_income = genetic_algorithm(population_size, mutation_rate, num_generations, customers)
print("Jalur terbaik:", best_chromosome)
print("Total pendapatan terbesar:", total_income)


Jalur terbaik: ['g', 'j', 'i', 'f', 'a', 'd', 'e', 'b', 'h', 'c']
Total pendapatan terbesar: 60000
